In [9]:
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from shapely.geometry import Point
import pandas as pd

In [10]:
# Categorize time labels
def categorize_time(time_str):
    tm = datetime.strptime(time_str, '%H:%M:%S').time()
    if tm.hour < 6:
        return 'Late Night'
    elif 6 <= tm.hour < 12:
        return 'Morning'
    elif 12 <= tm.hour < 18:
        return 'Afternoon'
    elif 18 <= tm.hour:
        return 'Evening'

# Fix the pricing format issues
def fix_price(price):
    return price.replace('$', '').replace(',', '')

In [11]:
# Load in data
df = pd.read_csv(r'C:\Users\15179\Desktop\CSE881 Project\Data\Airbnb_Open_Data.csv', low_memory=False)

# Specify the columns of interest
columns_of_interest = ['neighbourhood group', 'lat', 'long', 'price', 'host_identity_verified', 'instant_bookable', 'review rate number']

# Drop rows with NaN values in specified columns
df = df.dropna(subset=columns_of_interest)
df = df[columns_of_interest]

# Fix row in neighbourhood group that has 'brookln' instead of 'brooklyn'
df['neighbourhood group'] = df['neighbourhood group'].replace('brookln', 'Brooklyn')
df['neighbourhood group'].value_counts()

# Fix the pricing format issues
df['price'] = df['price'].apply(fix_price)
df['price'] = df['price'].astype(float)

# Convert `host_identity_verified` to categorical column then convert to binomial (0 & 1)
df['host_identity_verified'] = df['host_identity_verified'].astype('category')
df['host_identity_verified'] = df['host_identity_verified'].cat.codes

# Convert `instant_bookable` to 0 and 1 categorical column
df['instant_bookable'] = df['instant_bookable'].astype('category')
df['instant_bookable'] = df['instant_bookable'].cat.codes

# Drop columns where value counts is equal to 1
for col in df.columns:
    if df[col].value_counts().shape[0] == 1:
        df = df.drop(col, axis=1)
        
df_airbnb = df.copy()

In [12]:
# Import .shp file of Borough NY
street_map = gpd.read_file(r'C:\Users\15179\Desktop\CSE881 Project\Map\geo_export_336838c1-1789-4dd5-84ec-a3625918536c.shp')

In [13]:
import folium
from folium.plugins import MarkerCluster

def heatMapPage():
    global df_airbnb
    global axis_choice

    # Create a base map
    map_center = [df_airbnb['lat'].mean(), df_airbnb['long'].mean()]  # Use the mean coordinates as the initial center
    my_map = folium.Map(location=map_center, zoom_start=12)

    # Add MarkerCluster for Airbnb points
    marker_cluster = MarkerCluster().add_to(my_map)

    for index, row in df_airbnb.iterrows():
        # Create a popup with price information
        popup_text = f"Price: ${row['price']}"
        popup = folium.Popup(popup_text, parse_html=True)

        # Create a marker with the popup and add it to the MarkerCluster
        folium.Marker([row['lat'], row['long']], popup=popup).add_to(marker_cluster)

    # Display the map
    st.markdown("### NY Airbnb Map")
    st.write(my_map)

# Call the function
heatMapPage()

NameError: name 'st' is not defined